In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

num = 0
for dirname, _, filenames in os.walk('../data/raw/UCF50'):
    for filename in filenames:
        num +=1
print(num)
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

6681


In [2]:
# model.py
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout



2024-06-20 17:37:44.087213: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-20 17:37:44.087353: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-20 17:37:44.250951: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:


# Define function to load and preprocess UCF50 dataset
def load_ucf50_data(directory, sample_per_vid_passed = 10):
    
    classes = os.listdir(directory)
    num_classes = len(classes)
    
    class_to_label = {cls: i for i, cls in enumerate(classes)}
    X = []
    y = []
    
    # go through all folders (classes)
    for cls in classes:
        class_dir = os.path.join(directory, cls)
        
        # go through all video files
        for video_file in os.listdir(class_dir):
            
            # just make sure they are videos :>
            if video_file.endswith('.avi'):
                
                # initialize the counter
                sample_per_vid = 0
                
                video_path = os.path.join(class_dir, video_file)
                cap = cv2.VideoCapture(video_path)
                
                # go through all frames in that video
                
                while cap.isOpened():
                    ret, frame = cap.read()   
                    if not ret:
                            # ret is an indicator that there is a frame, if not ret, the vid has ended
                            #if sample_per_vid < 10:
                                # do some action :>
                                
                            
                        break
                    
                    '''
                    each vid, I take only 10 random frames 
                    '''
                    
                    if sample_per_vid > (sample_per_vid_passed - 1):
                        
                        break
                        
                    if np.random.randint(0,10) > 4:
                        
                        frame = cv2.resize(frame, (64, 64))
                        X.append(frame)
                        y.append(class_to_label[cls])
                        sample_per_vid += 1
                    
                        
                cap.release()
    X = np.array(X)
    y = np.array(y)
    
    # Calculate num_classes
    num_classes = len(np.unique(y))
    
    return X, y
    


In [4]:
X, y = load_ucf50_data('/kaggle/input/realistic-action-recognition-ucf50/UCF50')

In [5]:
len(os.listdir("/kaggle/input/realistic-action-recognition-ucf50/UCF50/HorseRace"))

127

In [6]:
y[0:1500]

array([0, 0, 0, ..., 1, 1, 1])

In [7]:
X.shape

(66810, 64, 64, 3)

In [8]:

# Compute num_classes
num_classes = len(np.unique(y))
print(len(X), len(y))


# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
'''
random seed
as the dataset is sorted based on the class [class 0 frames, class 1 frames, class 2 frames,...]
so I use random state parameter to randomize the process of spliting 

'''

# Normalize pixel values to range [0, 1]
# We only normalize the X
X_train = X_train / 255.0
X_test = X_test / 255.0


66810 66810


In [9]:
# Define CNN model
model = Sequential([
    
    # FEATURE EXTRACTOOOOORRRRRRR
    # convolution applies a filter
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    # reduces unnecassary details 
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    
    Flatten(),
    # with some features
    
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:

# Compile model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [11]:

# Train model - HYPERPARAMETERS
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
1337/1337 ━━━━━━━━━━━━━━━━━━━━ 95s 70ms/step - accuracy: 0.4590 - loss: 2.1062 - val_accuracy: 0.8997 - val_loss: 0.3528
Epoch 2/10
1337/1337 ━━━━━━━━━━━━━━━━━━━━ 92s 69ms/step - accuracy: 0.9360 - loss: 0.2245 - val_accuracy: 0.9580 - val_loss: 0.1587
Epoch 3/10
1337/1337 ━━━━━━━━━━━━━━━━━━━━ 149s 75ms/step - accuracy: 0.9773 - loss: 0.0843 - val_accuracy: 0.9680 - val_loss: 0.1199
Epoch 4/10
1337/1337 ━━━━━━━━━━━━━━━━━━━━ 100s 75ms/step - accuracy: 0.9821 - loss: 0.0596 - val_accuracy: 0.9691 - val_loss: 0.1203
Epoch 5/10
1337/1337 ━━━━━━━━━━━━━━━━━━━━ 98s 73ms/step - accuracy: 0.9855 - loss: 0.0509 - val_accuracy: 0.9745 - val_loss: 0.0986
Epoch 6/10
1337/1337 ━━━━━━━━━━━━━━━━━━━━ 96s 72ms/step - accuracy: 0.9911 - loss: 0.0316 - val_accuracy: 0.9573 - val_loss: 0.1686
Epoch 7/10
1337/1337 ━━━━━━━━━━━━━━━━━━━━ 95s 71ms/step - accuracy: 0.9895 - loss: 0.0347 - val_accuracy: 0.9511 - val_loss: 0.2003
Epoch 8/10
1337/1337 ━━━━━━━━━━━━━━━━━━━━ 95s 71ms/step - accuracy: 0.9911

In [12]:
# Evaluate model on test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", test_accuracy)

418/418 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 0.9728 - loss: 0.1169
Test Accuracy: 0.9724591970443726


In [13]:
#create the model and add the layers [Convolution Layers]

model_1 = Sequential()

#When working with emages, use 2D, for some kind of data like sound-waves you can use 1D
#32 felters, each one for detecting one pattern
#(3,3) the small window size ,adding padding is optional

model_1.add(Conv2D(32, (3,3), padding = 'same' ,activation= 'relu', input_shape =(64,64,3)))
model_1.add(Conv2D(32, (3,3), activation = 'relu')) # we wont pad the images 

model_1.add(Conv2D(64, (3,3), activation = 'relu', padding = 'same'))
model_1.add(Conv2D(64, (3,3), activation = 'relu'))

#tell keras, you no longer working with 2D data
model_1.add(Flatten())

model_1.add(Dense(512, activation = 'relu'))
model_1.add(Dense(50, activation = 'softmax'))

#print a summary of the model
model_1.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 64, 64, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 62, 62, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 62, 62, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 60, 60, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 230400)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │   117,965,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 50)             │        25,650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 118,056,530 (450.35 MB)

 Trainable params: 118,056,530 (450.35 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
#create the model and add the layers [Max pooling]

model_2 = Sequential()

model_2.add(Conv2D(32, (3,3), padding = 'same' ,activation= 'relu', input_shape =(64,64,3)))
model_2.add(Conv2D(32, (3,3), activation = 'relu')) # we wont pad the images 
model_2.add(MaxPooling2D(pool_size = (2,2)))

model_2.add(Conv2D(64, (3,3), activation = 'relu', padding = 'same'))
model_2.add(Conv2D(64, (3,3), activation = 'relu'))
model_2.add(MaxPooling2D(pool_size = (2,2)))

model_2.add(Flatten())

model_2.add(Dense(512, activation = 'relu'))
model_2.add(Dense(50, activation = 'softmax'))


#print a summary of the model
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,605,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 50)             │         6,450 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,894,808 (18.67 MB)

 Trainable params: 1,631,602 (6.22 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 3,263,206 (12.45 MB)

In [15]:
#create the model and add the layers [Dropout]

model_3 = Sequential()

model_3.add(Conv2D(32, (3,3), padding = 'same' ,activation= 'relu', input_shape =(64,64,3)))
model_3.add(Conv2D(32, (3,3), activation = 'relu')) # we wont pad the images 
model_3.add(MaxPooling2D(pool_size = (2,2)))
model_3.add(Dropout(0.25)) #0.25 to 0.50 works well

model_3.add(Conv2D(64, (3,3), activation = 'relu', padding = 'same'))
model_3.add(Conv2D(64, (3,3), activation = 'relu'))
model_3.add(MaxPooling2D(pool_size = (2,2)))
model_3.add(Dropout(0.25))

model_3.add(Flatten())

model_3.add(Dense(512, activation = 'relu'))
model_3.add(Dropout(0.50)) #make it suffer to learn

model_3.add(Dense(50, activation = 'softmax'))


#print a summary of the model
model_3.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_10 (Conv2D)              │ (None, 64, 64, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 62, 62, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 31, 31, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 29, 29, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 512)            │     6,423,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 50)             │        25,650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,514,258 (24.85 MB)

 Trainable params: 6,514,258 (24.85 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
model_3.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model_3.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
1337/1337 ━━━━━━━━━━━━━━━━━━━━ 337s 251ms/step - accuracy: 0.2837 - loss: 2.7904 - val_accuracy: 0.8584 - val_loss: 0.5340
Epoch 2/10
1337/1337 ━━━━━━━━━━━━━━━━━━━━ 385s 253ms/step - accuracy: 0.7839 - loss: 0.7508 - val_accuracy: 0.9537 - val_loss: 0.1736
Epoch 3/10
1337/1337 ━━━━━━━━━━━━━━━━━━━━ 339s 254ms/step - accuracy: 0.8793 - loss: 0.4070 - val_accuracy: 0.9695 - val_loss: 0.1093
Epoch 4/10
1337/1337 ━━━━━━━━━━━━━━━━━━━━ 383s 255ms/step - accuracy: 0.9117 - loss: 0.2884 - val_accuracy: 0.9731 - val_loss: 0.0959
Epoch 5/10
1337/1337 ━━━━━━━━━━━━━━━━━━━━ 378s 252ms/step - accuracy: 0.9299 - loss: 0.2264 - val_accuracy: 0.9833 - val_loss: 0.0634
Epoch 6/10
1337/1337 ━━━━━━━━━━━━━━━━━━━━ 339s 254ms/step - accuracy: 0.9417 - loss: 0.1857 - val_accuracy: 0.9869 - val_loss: 0.0528
Epoch 7/10
1337/1337 ━━━━━━━━━━━━━━━━━━━━ 335s 250ms/step - accuracy: 0.9479 - loss: 0.1718 - val_accuracy: 0.9876 - val_loss: 0.0472
Epoch 8/10
1337/1337 ━━━━━━━━━━━━━━━━━━━━ 335s 251ms/step - ac

In [17]:
for model in [model_1, model_2, model_3]:
    # Compile model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    # Train model - HYPERPARAMETERS
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
1337/1337 ━━━━━━━━━━━━━━━━━━━━ 1826s 1s/step - accuracy: 0.5284 - loss: 1.9463 - val_accuracy: 0.9378 - val_loss: 0.2272
Epoch 2/10
1337/1337 ━━━━━━━━━━━━━━━━━━━━ 1878s 1s/step - accuracy: 0.9742 - loss: 0.0935 - val_accuracy: 0.9394 - val_loss: 0.2327
Epoch 3/10
1337/1337 ━━━━━━━━━━━━━━━━━━━━ 1778s 1s/step - accuracy: 0.9820 - loss: 0.0595 - val_accuracy: 0.9579 - val_loss: 0.1827
Epoch 4/10
1337/1337 ━━━━━━━━━━━━━━━━━━━━ 1832s 1s/step - accuracy: 0.9873 - loss: 0.0447 - val_accuracy: 0.9485 - val_loss: 0.2451
Epoch 5/10
1337/1337 ━━━━━━━━━━━━━━━━━━━━ 1761s 1s/step - accuracy: 0.9890 - loss: 0.0377 - val_accuracy: 0.9539 - val_loss: 0.2499
Epoch 6/10
1337/1337 ━━━━━━━━━━━━━━━━━━━━ 1686s 1s/step - accuracy: 0.9933 - loss: 0.0260 - val_accuracy: 0.9370 - val_loss: 0.3119
Epoch 7/10
1337/1337 ━━━━━━━━━━━━━━━━━━━━ 1680s 1s/step - accuracy: 0.9928 - loss: 0.0238 - val_accuracy: 0.9458 - val_loss: 0.3298
Epoch 8/10
1337/1337 ━━━━━━━━━━━━━━━━━━━━ 1666s 1s/step - accuracy: 0.9941 -

In [18]:
for model,i in zip([model_1, model_2, model_3],[1,2,3]):
# Evaluate model on test set
    test_loss, test_accuracy = model.evaluate(X_test, y_test)
    print(f"Test Accuracy for model {i}:", test_accuracy)

418/418 ━━━━━━━━━━━━━━━━━━━━ 92s 219ms/step - accuracy: 0.9484 - loss: 0.3201
Test Accuracy for model 1: 0.9479119777679443
418/418 ━━━━━━━━━━━━━━━━━━━━ 24s 58ms/step - accuracy: 0.9744 - loss: 0.1310
Test Accuracy for model 2: 0.9735069870948792
418/418 ━━━━━━━━━━━━━━━━━━━━ 25s 60ms/step - accuracy: 0.9924 - loss: 0.0282
Test Accuracy for model 3: 0.9923664331436157


In [19]:
# confusion matrix for the 3 different models